<a href="https://colab.research.google.com/github/Mariolh33/G2_100472188_100429058/blob/main/Competicion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos desde github el modelo y los datos de competicion

In [ ]:
import pickle
import pandas as pd
import requests
from io import BytesIO
import gzip

# URL del archivo CSV comprimido
url_datos_competicion = 'https://github.com/Mariolh33/G2_100472188_100429058/raw/main/wind_comp.csv.gz'

# Descargar el archivo CSV comprimido
response = requests.get(url_datos_competicion)

# Verificar si la descarga fue exitosa
if response.status_code == 200:
    # Descomprimir el archivo y cargarlo en un DataFrame
    with gzip.open(BytesIO(response.content), 'rt') as file:
        datos_competicion = pd.read_csv(file)
else:
    print("Error al descargar el archivo.")

# URL del archivo pkl
url_modelo_final = 'https://github.com/Mariolh33/G2_100472188_100429058/raw/main/modelo_final.pkl'

# Descargar el archivo pkl
response_modelo = requests.get(url_modelo_final)

# Verificar si la descarga fue exitosa
if response_modelo.status_code == 200:
    # Cargar el modelo desde el contenido descargado
    modelo_entrenado = pickle.loads(response_modelo.content)
else:
    print("Error al descargar el archivo.")


Limpiamos los datos de competición de la misma forma que hicimos con los datos de entrenamiento anteriormente, de esta forma los datos de entreda seran los mismos con los que se entrenó el modelo.

Una vez limpios realizamos la predicción y procedemos a guardarlos en un archivo csv

In [ ]:

# Seleccionar las variables a mantener y eliminar
variables_a_eliminar = {'lai_lv.13', 'v100.13', 'v10.13', 'u10.13'}
columnas_a_mantener = ['datetime'] + [col for col in datos_competicion.columns if col.endswith('13') and col not in variables_a_eliminar]

# Filtrar el DataFrame para mantener solo las columnas seleccionadas
datos_finales = datos_competicion[columnas_a_mantener]



# Seleccionar solo los datos de entrenamiento
datos_finales = datos_finales.drop(['datetime'], axis=1)

# Usar el modelo para hacer predicciones en el conjunto de datos de entrenamiento
predicciones = modelo_entrenado.predict(datos_finales)

# Convertir las predicciones a un DataFrame
df_predicciones = pd.DataFrame(predicciones, columns=['Predicciones'])

# Guardar las predicciones en un archivo CSV
df_predicciones.to_csv('predicciones.csv', index=False)